# 3: Explore & Cluster Toronto Neighborhoods

<b>Challenge:</b> Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

Just make sure:

* To add enough Markdown cells to explain what you decided to do and to report any observations you make. 
* To generate maps to visualize your neighborhoods and how they cluster together. 

In [86]:
from bs4 import BeautifulSoup
import requests as rq

import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium==0.5.0
import folium
print('folim is installed!')

from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

print('Libraries are imported now!')

folim is installed!
Solving environment: done

# All requested packages already installed.

Libraries are imported now!


Let's webscrape the list of postal codes from the Wikipedia page and then create a dictionary with 3 Key fields.

In [87]:
url = rq.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'xml')

table_contents = []
table = soup.find('table')

#table

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

        
#table_contents
        

Let's create a new dataframe called Zip_df and fix some discrepancies in Borough data

In [88]:
Zip_df=pd.DataFrame(table_contents)
Zip_df['Borough']=Zip_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#Zip_df.head()
Zip_df.shape

(103, 3)

Create get_geocode() function and then read the CSV file from the web which include Latitude & Longitude information.

In [89]:
def get_geocode(zip_code):
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

# Assign geocode values from csv file to the dataframe
geo_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [90]:
# Rename field name to match with Zip_df dataframe
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace = True)

# Merge Zip_df data to geo_df using PostalCode field
geo_merged = pd.merge(geo_df, Zip_df, on = 'PostalCode')
# geo_merged

# Arrange fields and create TO_df new dataframe
TO_df = geo_merged[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
TO_df.head() #Final call for the second notebook

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Now, I will analyse data only for Scarborough

In [91]:
SB_df = TO_df[TO_df['Borough'] == 'Scarborough'].reset_index(drop = True)
SB_df.head(20) 
#There are only 17 records so we are good

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Define Foursquare Credentials & Version

In [92]:
CLIENT_ID = '2L0PQXFC0Z2OZQ2MBHTGAAPBAKNBCDKS5TMWJEGVJI1RO3AU'
CLIENT_SECRET = 'SUS0C4RU3KI2D4IHRDRT3KZW0OEDNS2T2U1AWX3JNNIQ2E5Q'
VERSION = '20180605'
LIMIT = 100

Let's create a function to explore all neighborhoods in Scarborough

In [93]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = rq.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's run above function on each neighborhood & create a new dataframe called SB_venues

In [94]:
SB_venues = getNearbyVenues(names=SB_df['Neighborhood'],
                                   latitudes=SB_df['Latitude'],
                                   longitudes=SB_df['Longitude']
                                  )
#SB_venues

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [95]:
SB_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


Let's check how many venues were returned for each neighborhood

In [96]:
SB_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


Let's see how many unique categories can be curated from all the returned venues

In [54]:
print('There are {} unique categories.'.format(len(SB_venues['Venue Category'].unique())))

There are 52 unique categories.


#### Let's Analyze Each Neighborhood

In [97]:
# One hot encoding
SB_onehot = pd.get_dummies(SB_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
SB_onehot['Neighborhood'] = SB_venues['Neighborhood'] 

# Move neighborhood column to the first column
fixed_columns = [SB_onehot.columns[-1]] + list(SB_onehot.columns[:-1])
SB_onehot = SB_onehot[fixed_columns]

SB_onehot.head()


,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Now, Let's group rows by neighborhood & by taking the mean of the frequency of occurrence of each category

In [98]:
SB_grouped = SB_onehot.groupby('Neighborhood').mean().reset_index()
SB_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000,0.000000,0.0,0.250000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.25,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.25,0.0,0.0,0.000000,0.0
2,Cedarbrae,0.0,0.125,0.0,0.125,0.125000,0.0,0.000000,0.0,0.0,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.125,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.0,0.0,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.000,0.076923,0.0,0.000000,0.0,0.0,0.00,0.000,0.076923,0.000000,0.00,0.076923,0.000000,0.0,0.000,0.076923,0.076923,0.076923,0.00,0.000000,0.000,0.0,0.0,0.0,0.000000,0.076923,0.000000,0.00,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.076923,0.000000,0.0,0.076923,0.153846,0.000000,0.000,0.000,0.000000,0.076923,0.00,0.0,0.0,0.076923,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.0,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.200000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.0,0.4,0.000000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.0,0.200,0.000000,0.0,0.000000,0.2,0.1,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.1,0.0,0.100000,0.000000,0.000000,0.00,0.00,0.000,0.1,0.000,0.0,0.0,0.000000,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.00,0.0,0.1,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.0,0.000,0.0,0.000,0.125000,0.0,0.125000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.125,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.0,0.0,0.0,0.125000,0.000000,0.000000,0.00,0.00,0.125,0.0,0.125,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.125,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0
8,"Kennedy Park, Ionview, East Birchmount Park",0.0,0.000,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000,0.200000,0.200000,0.00,0.200000,0.000000,0.2,0.000,0.000000,0.000000,0.0

#### Let's put new dataframe and display Top 10 venues for each neighborhood
But, first, let's write a function to sort the venues in descending order

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, let's create a new dataframe for Top 10 Venues

In [100]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SB_grouped['Neighborhood']

for ind in np.arange(SB_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SB_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Gym,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Gym,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Cosmetics Shop,Hobby Shop
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Bank,Convenience Store,Pharmacy,Italian Restaurant,Fast Food Restaurant,Shopping Mall,Fried Chicken Joint,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Let's create Clusters for Neighborhoods
Run K-means to cluster the Neighborhoods

In [101]:
# set number of clusters
kclusters = 5

SB_grouped_clustering = SB_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SB_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [103]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SB_merged = SB_df

# merge SB_merged with SB_df to add latitude/longitude for each neighborhood
SB_merged = SB_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

SB_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean BBQ Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Cosmetics Shop,Hobby Shop


We can see float values in 'Cluster Labels' field so let's convert it to integer

In [127]:
# In order to convert float to integer, we need to make sure that there is no NaN present
SB_merged['Cluster Labels'] = SB_merged['Cluster Labels'].replace(np.nan, 0)

SB_merged['Cluster Labels'] = SB_merged['Cluster Labels'].apply(np.int64)

SB_merged = SB_merged.drop(['PostalCode'], axis=1)

SB_merged.head()
#SB_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store
3,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Cosmetics Shop,Hobby Shop


#### Let's visualize the resulting clusters 
But, first, let's define geographical coordinates of Scarborough

In [128]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


Now, let's visualize clusters in map

In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SB_merged['Latitude'], SB_merged['Longitude'], SB_merged['Neighborhood'], SB_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's Examine Clusters

#### Cluster: 1

In [130]:
SB_merged.loc[SB_merged['Cluster Labels'] == 1, SB_merged.columns[[1] + list(range(5, SB_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Department Store


#### Cluster: 2

In [131]:
SB_merged.loc[SB_merged['Cluster Labels'] == 2, SB_merged.columns[[1] + list(range(5, SB_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster: 3

In [132]:
SB_merged.loc[SB_merged['Cluster Labels'] == 3, SB_merged.columns[[1] + list(range(5, SB_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,Coffee Shop,Korean BBQ Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster: 4

In [133]:
SB_merged.loc[SB_merged['Cluster Labels'] == 4, SB_merged.columns[[1] + list(range(5, SB_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster: 5

In [134]:
SB_merged.loc[SB_merged['Cluster Labels'] == 5, SB_merged.columns[[1] + list(range(5, SB_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Nothing assigned to #5 cluster